In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import src.SessionNavigator as SessionNavigator
import src.SessionProcessor as SessionProcessor
from sklearn.svm import LinearSVC
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

C:\Users\Demogorgon\anaconda3\envs\Python37\lib\site-packages\allensdk\brain_observatory\comparison_utils.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
# Do you want the plots saved?
SAVE = False

# Insert the file path to your data (e.g. "C:\Users\MickeyMouse\MouseBrainData\manifest.json")
data_root = "C:/Users/Demogorgon/Documents/College/Marcus/Boston University PhD/Ocker Lab"
manifest_path = f"{data_root}/AllenSDK_Data/manifest.json"
save_path = f"{data_root}/correlations_and_bursts/data"
bursts_path = f"{save_path}/"

# These are the same stim and session_id in the Burst Detection script
stim = 'drifting_gratings_contrast'
session_id = 778240327

In [3]:
# Create a SessionNavigator, which will open and navigate the data in manifest_path
navigator = SessionNavigator.SessionNavigator(manifest_path)

# Load the session and open a SessionProcessor
session = navigator.load_session(session_id)
processor = SessionProcessor.SessionProcessor(session)

In [4]:
session_name = processor.construct_decoder('drifting_gratings_contrast', 'orientation')
processor.construct_psth(session_name)
processor.calculate_decoder_weights(session_name)
processor.calculate_correlations(session_name)

In [5]:
with open(f"{save_path}/{stim}__whole_burst_trains__session_{session_id}.pkl", 'rb') as f:
    bursts = pkl.load(f)
with open(f"{save_path}/{stim}__whole_single_trains__session_{session_id}.pkl", 'rb') as f:
    singles = pkl.load(f)
processor.add_bursts(bursts, singles, session_name, shuffled=False)
presentationwise_bursts = processor.presentationwise_burst_count(session_name)
presentationwise_singles= processor.presentationwise_non_burst_count(session_name)

In [6]:
results = processor.results()['drifting_gratings_contrast__width_50.0ms__unshuffled']
decoder = results['decoder']
bins = decoder.bin_edges
unit_ids = processor.all_units
stim_presentation_ids = decoder.stim_table.index.values
#presentationwise_example = results['psths']
burst_example = bursts[951012296]
presentationwise_example = session.presentationwise_spike_times(stimulus_presentation_ids=stim_presentation_ids, unit_ids=unit_ids)

In [7]:
presentationwise_burst_example = processor.presentationwise_burst_times(session_name)
presentationwise_single_example= processor.presentationwise_non_burst_times(session_name)
# TODO: Decoding from isolated spikes and from bursts (as expected)
# TODO: 2 channel code, a unit's isolated spikes and a unit's bursts (so num_features will double)
# NEED: Controls to account for the doubling of the features
# Random sample from 50% of the population to account for the doubling of features (repeat for many draws before comparing)

# Finally, compare regular vs trial shuffled

# NEED: Loads of figures, show raw and separated spike trains in the same plot
# Next Monday: Have a practice run